In [1]:
import os, shutil, re
import pandas as pd
from helpers import *
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from scipy.signal import resample
import librosa
import librosa.display
from pydub import AudioSegment
from datetime import datetime
from helpers import *
%matplotlib inline

In [2]:
# replace this with your root directory
ROOT = os.getcwd() + "/openbci_data/"
AUD_ROOT = os.getcwd() + "/audio/no_voice/"
pattern = "[0-9]{2}_[0-9]{2}"

"""
Expected directory structure:
[INSIDE ROOT DIRECTORY]
---- [category] voice, no_voice
-------- [date] 07_02, 07_09, ...
------------ [label] down, go, ...
---------------- [channel] ch1, ch2, ...
-------------------- [wave files] *.wav
"""
# VARS 

VALID_LABELS = ["yes", "no", "stop", "go", "right", "left", "down", "up", "on", "off", "test"]
USELESS = ["ch2", "ch4", "na1", "na2", "na3"]
IMG_EXT = ".png"
VERBOSITY = 1000

In [3]:
# def write_wav(data, filename, framerate, amplitude):
#     wavfile = wave.open(filename, 'w')
#     nchannels = 1
#     sampwidth = 2
#     framerate = framerate
#     nframes = len(data)
#     comptypte = "NONE"
#     compname = "not compressed"
#     wavfile.setparams((nchannels,
#                       sampwidth,
#                       framerate,
#                       nframes,
#                       comptype,
#                       compname))
#     frames = []
#     for s in data:
#         mul = int(s * amplitude)
#         frames.append(struct.pack('h', mul))
#     frames = ''.join(frames)
#     wavfile.writeframes(frames)
#     wavfile.close()

In [4]:
def timestamp2milsec(timestamp):
    timesplit = timestamp.split('.')
    if len(timesplit) == 2:
        milsec = int(timesplit[1])
        time = timesplit[0].split(':')
        time_milsec = int(time[0]) * 60 * 60 * 1000 + int(time[1]) * 60 * 1000 + int(time[2]) * 1000 + milsec
        return time_milsec

In [5]:
# to do: make it so we can change label number dynamically

def make_image(channel_data, ch_name): 
    channel = channel_data[ch_name]
    time = range(400)
    samples = len(channel) // 400
    x_counter = 0
    y_counter = 0
    i = 0
    while i < samples:
        curr = channel.iloc[:400]
        fig = plt.figure(figsize=(1.28, 1.28), dpi=100, frameon=False)
        plt.plot(time, curr)
        plt.ylim(-700, 700)
        fig.subplots_adjust(bottom = 0)
        fig.subplots_adjust(top = 1)
        fig.subplots_adjust(right = 1)
        fig.subplots_adjust(left = 0)
        plt.axis('off')
        if i % 2 == 0:
            fig.savefig(IMG_ROOT + "/no/" + ch_name + "/" + str(x_counter).zfill(4) + IMG_EXT, pad_inches=0)
            x_counter += 1
        else:
            fig.savefig(IMG_ROOT + "/yes/" + ch_name + "/" + str(y_counter).zfill(4) + IMG_EXT, pad_inches=0)
            y_counter += 1
        plt.close()
        channel = channel.iloc[400:]
        i += 1

In [6]:
# read csv(txt) file write to another csv file to convert into wav files

df = pd.read_csv("./openbci_raw_09_22/09_22_21/09_22_21.txt", ",")
df = df.drop(columns=USELESS)
df = df.dropna()
ch1_df = df['ch1'].tolist()
# ch3_df = df['ch3'].tolist()

# df['timestamp'] = df['timestamp'].map(timestamp2milsec)
# df['timestamp'] = df['timestamp'].map(lambda x: x - df['timestamp'].iloc[0])

df.tail()

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Id,ch1,ch3,timestamp
419591,106,221.97,8.38,22:49:30.312
419592,107,213.42,7.94,22:49:30.326
419593,108,259.17,11.52,22:49:30.326
419594,109,235.83,4.29,22:49:30.328
419595,110,194.61,10.09,22:49:30.328


In [10]:
def channel2wav(ch_data, fname):
    data = np.array(ch_data, dtype='float64')
    data /= np.max(np.abs(data))
    data_resampled = resample(data, len(data))
    wavfile.write(fname + '.wav', 200, data_resampled)
    print("wav file conversion done")

In [12]:
channel2wav(ch1_df, 'ch1')
# channel2wav(ch3_df, 'ch3')

wav file conversion done
